In [42]:
# import libraries
import cv2
import numpy as np
import mediapipe as mp
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout, BatchNormalization, TimeDistributed, Lambda, GlobalMaxPooling3D
from keras.applications import VGG19
from keras.callbacks import TensorBoard

In [2]:
mp_holistic = mp.solutions.holistic # holistic model
mp_drawing = mp.solutions.drawing_utils # Initialize mediapipe drawing utilities

In [3]:
def detection (img, model):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # convert the input into RGB format to pass to mediapipe
    img.flags.writeable = False # To improve performance, mark the image as not writeable
    results = model.process(img) # detection model
    img.flags.writeable = True
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # convert the input into BGR format to pass to opencv imshow
    return img, results

In [4]:
def draw_landmarks (img, results):
    mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,128,0), thickness=2, circle_radius=2),mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(img, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,51,153), thickness=2, circle_radius=2),mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(img, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,51,153), thickness=2, circle_radius=2),mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2))

In [5]:
# get webcam
cap = cv2.VideoCapture('./videoplayback.mp4')
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while True:
        ret, frame = cap.read()

        img, results = detection(frame, holistic) # make detectionq
        draw_landmarks(img,results) # draw landmarks

        cv2.imshow('Human Pose Estimation', img)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [6]:
# extracting body keypoints
# flatten the array so that it can be passed to the LSTM model
def keypoint_extraction(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, left_hand, right_hand])

In [7]:
DATA_PATH = os.path.join('data') 
actions = ['punch', 'guard', 'normal'] # declare actions
vids_num = 30
vid_length = 30 #frames

# data directory
for action in actions:
    for vid in range(vids_num):
        os.makedirs(os.path.join('data', action, str(vid)))

In [8]:
# collecting data
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for vid in range(vids_num):
            for frame_num in range(vid_length):
                ret, frame = cap.read()

                img, results = detection(frame, holistic)
                draw_landmarks(img,results) 

                if frame_num == 0: 
                    cv2.putText(img, 'Start collecting', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(img, 'Collecting frames for {} Video Number {}'.format(action, vid), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', img)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(img, 'Collecting frames for {} Video Number {}'.format(action, vid), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', img)
                
                keypoints = keypoint_extraction(results)
                npy_path = os.path.join(DATA_PATH, action, str(vid), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

In [22]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'punch': 0, 'guard': 1, 'normal': 2}

In [30]:
# save each frame in a numpy array
sequences, labels = [], []
for action in actions:
    for vid in range(vids_num):
        window = []
        for frame_num in range(vid_length):
            res = np.load(os.path.join(DATA_PATH, action, str(vid), "{}.npy".format(frame_num)))
            res = cv2.resize(res,(256,256))
            res = res/255.0 #normalization

            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [31]:
print(np.array(sequences).shape)
print(np.array(labels).shape)

(90, 30, 258)
(90,)


In [33]:
# dataset
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)
print('X_train shape: ', X_train.shape)
print('y_train shape: ',y_train.shape)
print('X_test shape: ',X_test.shape)
print('y_test shape: ',y_test.shape)

X_train shape:  (85, 30, 258)
y_train shape:  (85, 3)
X_test shape:  (5, 30, 258)
y_test shape:  (5, 3)


In [57]:
# LSTM model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258)))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [58]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 64)            82688     
                                                                 
 dropout_6 (Dropout)         (None, 30, 64)            0         
                                                                 
 lstm_7 (LSTM)               (None, 30, 128)           98816     
                                                                 
 dropout_7 (Dropout)         (None, 30, 128)           0         
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_8 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 64)               

In [61]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
3/3 [==============================] - 1s 21ms/step - loss: 0.4398 - accuracy: 0.7529
Epoch 2/100
3/3 [==============================] - 0s 23ms/step - loss: 0.4599 - accuracy: 0.6941
Epoch 3/100
3/3 [==============================] - 0s 21ms/step - loss: 0.4640 - accuracy: 0.6588
Epoch 4/100
3/3 [==============================] - 0s 22ms/step - loss: 0.4608 - accuracy: 0.6824
Epoch 5/100
3/3 [==============================] - 0s 33ms/step - loss: 0.4400 - accuracy: 0.7765
Epoch 6/100
3/3 [==============================] - 0s 23ms/step - loss: 0.4246 - accuracy: 0.7294
Epoch 7/100
3/3 [==============================] - 0s 29ms/step - loss: 0.4661 - accuracy: 0.6824
Epoch 8/100
3/3 [==============================] - 0s 34ms/step - loss: 0.4294 - accuracy: 0.7647
Epoch 9/100
3/3 [==============================] - 0s 22ms/step - loss: 0.4669 - accuracy: 0.6824
Epoch 10/100
3/3 [==============================] - 0s 20ms/step - loss: 0.4468 - accuracy: 0.7412
Epoch 11/100
3/3 [=

In [62]:
# predictions
res = model.predict(X_test)
print(actions[np.argmax(res[4])])
print(actions[np.argmax(y_test[4])])

1/1 [==============================] - 0s 411ms/step
punch
punch


In [ ]:
model.save('action.h5')